In [1]:
import os, glob, shutil, argparse, re
import pandas as pd
from pathlib import Path
import numpy as np
import training_info


# parser = argparse.ArgumentParser()
# parser.add_argument('--batch', type=int, default=4)
# parser.add_argument('--all', action='store_tru e', default=False)
# args = parser.parse_args()
# dir_pattern = './log/pure_dataloading_b{}_worker{}_epoch1_iter10/'
logdir=training_info.logdir
dir_pattern = logdir+'/{}/{}/{}/{}/epoch{}/b{}/worker{}/thread{}/'
aug_pattern = logdir+'/{}/{}/'
simple_dir_pattern = logdir+'/{}'
logtypes=training_info.logtypes
datasets=training_info.datasets
models=training_info.models
term=training_info.term
output_file="output.log"

In [2]:
def get_dirname():
    dir_names = {}
    for logtype in logtypes:
        dir_names[logtype]=glob.glob(simple_dir_pattern.format(logtype)+f"/*/*/*/*/*/*/*/")

    return dir_names

def replace_str(target):
    target=target.replace('\n', '')
    target=target.replace(',', '')
    return target

def find_value(arr, target, jumpto=1):
    try:
        num=replace_str(arr[arr.index(target)+jumpto])
    except:
        print(arr, target, arr[arr.index(target)+jumpto])
        num='NA'
    return num

parsed_dir=training_info.parsed_dir
os.makedirs(parsed_dir, exist_ok=True)

breakdown_col_name=["Epoch","Step","Iteration time (sec)","Training stall time (sec)","Throughput (image/sec)"]
dir_names=get_dirname()
print(dir_names)

{'train_only': ['./log/DDP/train_only/1024batch10/default/resnet18/epoch5/b1024/worker4/thread0/', './log/DDP/train_only/1024batch/default/resnet18/epoch5/b1024/worker2/thread0/', './log/DDP/train_only/1024batch/default/resnet18/epoch5/b1024/worker24/thread0/', './log/DDP/train_only/1024batch/default/resnet18/epoch5/b256/worker24/thread0/'], 'loadNtrain': ['./log/DDP/loadNtrain/size5/default/resnet18/epoch5/b256/worker2/thread0/', './log/DDP/loadNtrain/1024batch10/default/resnet18/epoch5/b1024/worker2/thread0/', './log/DDP/loadNtrain/1024batch10/default/resnet18/epoch5/b1024/worker24/thread0/', './log/DDP/loadNtrain/1024batch10/default/resnet18/epoch5/b256/worker24/thread0/', './log/DDP/loadNtrain/1024batch/default/resnet18/epoch5/b1024/worker2/thread0/'], 'prepNloadNtrain': ['./log/DDP/prepNloadNtrain/size5/default/resnet18/epoch5/b1024/worker4/thread0/', './log/DDP/prepNloadNtrain/size5/default/resnet18/epoch5/b1024/worker16/thread0/', './log/DDP/prepNloadNtrain/size5/default/resnet1

In [3]:
def process_prep(prepro_time, breakdown_parsed_log, data_idx, epoch_idx):
    prepro_time=prepro_time.replace("DataPreprocessingTime:","")
    if(data_idx<epoch_idx):
        breakdown_parsed_log[data_idx].append(prepro_time)
    else:
        breakdown_parsed_log.append([prepro_time])
    return data_idx+1

def parser(dir_names):
    for logtype in logtypes:
        for logdir in dir_names[logtype]:
            if not os.path.exists(logdir):
                continue
            logdir_list=logdir.split('/')
            dataset=find_value(logdir_list, logtype)
            aug=find_value(logdir_list, dataset)
            model=find_value(logdir_list, aug)
            epoch=find_value(logdir_list, model)
            batchsize=find_value(logdir_list, epoch)
            worker=find_value(logdir_list, batchsize)

            epoch_num=epoch.replace("epoch","")
            batchsize_num=batchsize.replace("b","")
            worker_num=worker.replace("worker","")
                             
            parse_filename=f"{logtype}_{dataset}_{model}_{aug}_{worker}_{epoch}_{batchsize}"
            
            logfile=logdir +"/"+ output_file
            breakdown_parsed_log = []

            data_idx=0
            epoch_idx=0
            
            for line in open(logfile, 'r').readlines():
                if line.startswith(f'Epoch:'): # start log
                    replace_txt=line.replace('\t',' ')
                    test =replace_txt.split(' ')
                    info = list(filter(lambda x: x != "", test))
#                         print(info)

                    epochNstep=find_value(info, "Epoch:")
                    epoch=epochNstep.split(']')[0].replace('[','')
                    if epochNstep.find('/') != -1:
                        step=epochNstep.split('[')[2].split('/')[0]
                    else:
                        step=find_value(info, "Epoch:",2).split('/')[0]

                    iter_time=find_value(info, "Time") # epoch avg: total n iter
                    data_time=find_value(info, "Data")# data time
                    throughput=find_value(info, "Throughput")# throughput avg
                    cur_log_info=[epoch,step,iter_time,data_time,throughput]
#                         print(cur_log_info)
                    breakdown_parsed_log.append(cur_log_info)
                else:
                    pass

            breakdown_df = pd.DataFrame(breakdown_parsed_log,
                              columns=breakdown_col_name)
            breakdown_df.dropna().to_csv(parsed_dir+"/"+parse_filename+".csv", sep=',', na_rep='NA')

In [4]:
parser(dir_names)

In [5]:
dataset_col_name=["Log type","Dataset size(GB)","Augmentation","Worker","Epoch","Batch size","Avg iteration time (sec)" ,"Avg training stall time (sec)", "Avg throughput (images/sec)","Avg filtered first 4 iteration epoch time (sec)", "Avg epoch time (sec)"]

def total_summary():
    total_log=[]
    for logtype in logtypes:
            for logdir in dir_names[logtype]:
                if not os.path.exists(logdir):
                    continue
                logdir_list=logdir.split('/')
                print(logdir_list)
                dataset=find_value(logdir_list, logtype)
                aug=find_value(logdir_list, dataset)
                model=find_value(logdir_list, aug)
                epoch=find_value(logdir_list, model)
                batchsize=find_value(logdir_list, epoch)
                worker=find_value(logdir_list, batchsize)
                print(batchsize)
                epoch_num=epoch.replace("epoch","")
                batchsize_num=batchsize.replace("b","")
                worker_num=worker.replace("worker","")
                
                parse_filename=f"{logtype}_{dataset}_{model}_{aug}_{worker}_{epoch}_{batchsize}"
                df=pd.read_csv(f"./{parsed_dir}/{parse_filename}.csv",index_col=None)
                print(parse_filename)
                
                df=df[df["Epoch"] > 0]
                avg_epoch_time = round(df["Iteration time (sec)"].astype(float).sum()/(int(epoch_num)-1),2)

                df=df[df["Step"] > 4]
                filtered_avg_epoch_time = round(df["Iteration time (sec)"].astype(float).sum()/(int(epoch_num)-1),2)
                print(df.describe())
                print("\n\n")
#                 print(parse_filename, df["Iteration time (sec)"])
                iter_avg=f'{round(df["Iteration time (sec)"].mean(),4)}±{round(df["Iteration time (sec)"].std(),2)}'
                data_avg=f'{round(df["Training stall time (sec)"].mean(),4)}±{round(df["Training stall time (sec)"].std(),2)}'
                throughput_avg=f'{round(df["Throughput (image/sec)"].mean(),2)}±{round(df["Throughput (image/sec)"].std(),2)}'

                total_log.append([term[logtype],term[dataset],term[aug],worker_num,epoch_num,batchsize_num,iter_avg,data_avg,throughput_avg,filtered_avg_epoch_time,avg_epoch_time])
                
    avg_df = pd.DataFrame(total_log,
                              columns=dataset_col_name)
    avg_df.to_csv(parsed_dir+f"/df_analyzer_total_summary.csv", sep=',', na_rep='NA')
    
total_summary()

['.', 'log', 'DDP', 'train_only', '1024batch10', 'default', 'resnet18', 'epoch5', 'b1024', 'worker4', 'thread0', '']
b1024
train_only_1024batch10_resnet18_default_worker4_epoch5_b1024
       Unnamed: 0 Epoch Step Iteration time (sec) Training stall time (sec)  \
count           0     0    0                    0                         0   
unique          0     0    0                    0                         0   
top           NaN   NaN  NaN                  NaN                       NaN   
freq          NaN   NaN  NaN                  NaN                       NaN   

       Throughput (image/sec)  
count                       0  
unique                      0  
top                       NaN  
freq                      NaN  



['.', 'log', 'DDP', 'train_only', '1024batch', 'default', 'resnet18', 'epoch5', 'b1024', 'worker2', 'thread0', '']
b1024
train_only_1024batch_resnet18_default_worker2_epoch5_b1024
        Unnamed: 0        Epoch         Step  Iteration time (sec)  \
count  

       Unnamed: 0       Epoch        Step  Iteration time (sec)  \
count  640.000000  640.000000  640.000000            640.000000   
mean   542.000000    2.500000   24.500000              3.549864   
std    207.923183    1.118908   11.552425              0.155852   
min    185.000000    1.000000    5.000000              3.260442   
25%    363.500000    1.750000   14.750000              3.431817   
50%    542.000000    2.500000   24.500000              3.574893   
75%    720.500000    3.250000   34.250000              3.653858   
max    899.000000    4.000000   44.000000              3.909378   

       Training stall time (sec)  Throughput (image/sec)  
count                 640.000000              640.000000  
mean                    2.090509               72.255178  
std                     1.117734                3.191293  
min                     0.018449               65.484000  
25%                     1.148182               70.063000  
50%                     2.333000          

       Unnamed: 0       Epoch       Step  Iteration time (sec)  \
count  352.000000  352.000000  352.00000            352.000000   
mean   326.000000    2.500000   15.50000              0.403837   
std    124.802842    1.119625    6.35332              0.139465   
min    113.000000    1.000000    5.00000              0.282620   
25%    219.500000    1.750000   10.00000              0.315509   
50%    326.000000    2.500000   15.50000              0.332234   
75%    432.500000    3.250000   21.00000              0.472273   
max    539.000000    4.000000   26.00000              0.941933   

       Training stall time (sec)  Throughput (image/sec)  
count                 352.000000              352.000000  
mean                    0.035415              691.485849  
std                     0.088366              174.059168  
min                     0.000040              271.782000  
25%                     0.000210              542.064000  
50%                     0.000393              770.5

       Unnamed: 0       Epoch       Step  Iteration time (sec)  \
count  352.000000  352.000000  352.00000            352.000000   
mean   326.000000    2.500000   15.50000              1.484441   
std    124.802842    1.119625    6.35332              0.021352   
min    113.000000    1.000000    5.00000              1.418883   
25%    219.500000    1.750000   10.00000              1.471702   
50%    326.000000    2.500000   15.50000              1.486311   
75%    432.500000    3.250000   21.00000              1.496119   
max    539.000000    4.000000   26.00000              1.537184   

       Training stall time (sec)  Throughput (image/sec)  
count                 352.000000              352.000000  
mean                    0.647506              172.491284  
std                     0.443042                2.493617  
min                     0.048945              166.538000  
25%                     0.198792              171.109250  
50%                     0.636516              172.2

In [6]:
dataset_col_name=["Log type","Dataset size(GB)","Augmentation","Worker","Epoch","Batch size","Median iteration time (sec)", "Median training stall time (sec)", "Median throughput (images/sec)"]

def total_midian_summary():
    total_log=[]
    for logtype in logtypes:
            for logdir in dir_names[logtype]:
                if not os.path.exists(logdir):
                    continue
                logdir_list=logdir.split('/')
                
                dataset=find_value(logdir_list, logtype)
                aug=find_value(logdir_list, dataset)
                model=find_value(logdir_list, aug)
                epoch=find_value(logdir_list, model)
                batchsize=find_value(logdir_list, epoch)
                worker=find_value(logdir_list, batchsize)
                
                epoch_num=epoch.replace("epoch","")
                batchsize_num=batchsize.replace("b","")
                worker_num=worker.replace("worker","")
                
                parse_filename=f"{logtype}_{dataset}_{model}_{aug}_{worker}_{epoch}_{batchsize}"
                df=pd.read_csv(f"./{parsed_dir}/{parse_filename}.csv",index_col=None)
                df=df[df["Step"] > 4]
                df=df[df["Epoch"] > 0]
#                 print(parse_filename, df["Iteration time (sec)"])
                iter_avg=df["Iteration time (sec)"].median()
                data_avg=df["Training stall time (sec)"].median()
                throughput_avg=df["Throughput (image/sec)"].median()

                total_log.append([term[logtype],term[dataset],term[aug],worker_num,epoch_num,batchsize_num,iter_avg,data_avg,throughput_avg])
                
    avg_df = pd.DataFrame(total_log,
                              columns=dataset_col_name)
    avg_df.to_csv(parsed_dir+f"/df_analyzer_total_median_summary.csv", sep=',', na_rep='NA')
    
total_midian_summary()